In [1]:
import backtester272 as bt
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')

# Load data

In [ ]:
def load_data(name):
    data = pd.read_csv(f'data/{name}.csv')
    data['Date'] = pd.to_datetime(data['Date'])
    data.set_index('Date', inplace=True)
    data = data.sort_index()
    return data

weights = load_data('weights')  
sectors = load_data('sectors')


In [3]:
# Initialisation du backtester
wip = bt.Backtester(sectors, benchmark_weights=weights)

In [4]:
start_date = sectors.index[0].strftime('%Y-%m-%d')
end_date = sectors.index[-1].strftime('%Y-%m-%d')

# Initialisation des paramètres globaux
global_params = {
    'start_date': start_date,
    'end_date': end_date,
    'freq': 30 * 6, # Fréquence de rebalancement de la stratégie MaxSharpe (6 mois)
    'window': 30 * 24, # Fenêtre de calcul de la stratégie MaxSharpe (2 ans)
    'freq_tactical': 30, # Fréquence de rebalancement de la stratégie tactique (1 mois)
    'window_tactical': 30 * 3, # Fenêtre de calcul de la stratégie tactique (3 mois)
    'freq_macro': 30 * 3, # Fréquence de rebalancement de la stratégie macro (3 mois)
    'aum': 100,
    'transaction_cost': 0.0005 # COût de transaction de 0.05%
}

stategy_constraints = { # Contraintes de la stratégie MaxSharpe
    'max_weight': 0.35,
    'min_weight': 0.05,
    'risk_free_rate': 0.02,
    'total_exposure': 1.0,
    'max_turnover': 1.5,
    'max_tracking_error': 0.07,
    'lmd_ridge':0
}

tactical_constraints = {
    'delta': 30,
    'nb_fractile':3, # Nombre de fractiles pour la stratégie tactique
    'alpha': 0.3
}

macro_constraints = {
    'alpha': 0.01,
    'window_size': 30 * 12, # Fenêtre de calcul de la stratégie macro (12 mois)
    'threshold': 0,
}

MSM = wip.run(**global_params, 
              strategy=bt.MaxSharpeStrategy(**stategy_constraints), 
              macro=bt.MacroTactical(**macro_constraints), 
              tactical=bt.RankMomentumTactical(**tactical_constraints))

|                              | Benchmark   | Tactical Benchmark   | Macro Benchmark   | Tactical Macro Benchmark   | MaxSharpeStrategy   | MaxSharpeStrategy Tactical   | MaxSharpeStrategy Macro   | MaxSharpeStrategy Tactical Macro   |
|:-----------------------------|:------------|:---------------------|:------------------|:---------------------------|:--------------------|:-----------------------------|:--------------------------|:-----------------------------------|
| Performance                  | 130.01%     | 145.72%              | 132.63%           | 153.24%                    | 143.66%             | 148.06%                      | 143.66%                   | 159.29%                            |
| CAGR                         | 7.50%       | 8.11%                | 7.60%             | 8.40%                      | 8.03%               | 8.20%                        | 8.03%                     | 8.62%                              |
| Volatility                   | 15.71%      | 15.91

,Performance,CAGR,Volatility,Max Drawdown,Sharpe Ratio,Win Rate,Average Win,Average Loss,Tracking Error
Benchmark,130.01%,7.50%,15.71%,-34.97%,0.54,75.00%,14.28%,-10.23%,0.00%
Tactical Benchmark,145.72%,8.11%,15.91%,-35.05%,0.57,75.00%,15.03%,-9.92%,1.11%
Macro Benchmark,132.63%,7.60%,15.46%,-34.88%,0.55,75.00%,14.38%,-10.06%,1.10%
Tactical Macro Benchmark,153.24%,8.40%,15.69%,-34.83%,0.59,75.00%,15.37%,-9.73%,1.38%
MaxSharpeStrategy,143.66%,8.03%,15.74%,-32.18%,0.57,75.00%,14.05%,-8.26%,4.36%
MaxSharpeStrategy Tactical,148.06%,8.20%,15.91%,-31.81%,0.58,66.67%,16.30%,-6.53%,4.05%
MaxSharpeStrategy Macro,143.66%,8.03%,15.74%,-32.18%,0.57,75.00%,14.05%,-8.26%,4.57%
MaxSharpeStrategy Tactical Macro,159.29%,8.62%,15.75%,-30.62%,0.60,66.67%,16.78%,-6.31%,4.38%


In [5]:
MSM.visualize() # Visualisation de la performance de la stratégie
MSM.gather_metrics(highlight_extremes=True) # Récupération des métriques de performance de la stratégie